# Абстрактивная суммаризация

### Контекст

Необходимо провести суммаризацию полученной информации методами абстрактивной сумммаризации.

Абстрактивная суммаризация – это метод автоматического реферирования текста, при котором создается новое краткое изложение с использованием перефразирования и генерации новых предложений, а не просто извлечения фрагментов из исходного текста.

### Что нужно сделать?

Попробовать разыне методы абстрактивной суммаризации:

- T5 (Text-to-Text Transfer Transformer) - https://github.com/google-research/text-to-text-transfer-transformer
- BART https://huggingface.co/docs/transformers/model_doc/bart

### Результат:

- код .py, .ipynb
- выводы

### Загрузка библиотек и блок используемых функций

In [1]:
# Импорт библиотек
import os
import pandas as pd
import numpy as np

!pip install sentencepiece --quiet
!pip install protobuf --quiet
!pip install --upgrade protobuf
from transformers import (
    MT5ForConditionalGeneration, 
    T5Tokenizer, 
    MBartForConditionalGeneration, 
    MBart50TokenizerFast,
    AutoTokenizer,
    AutoModelForSeq2SeqLM, 
    AutoModelForCausalLM)

import warnings
warnings.filterwarnings("ignore")

# Для метрик
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt

In [2]:
# Загрузка csv полученного из парсинга json и удаленеие строк, в которых столбец text пустой

def load_df(file, index=None):
    pth1 = os.path.join('data', 'example', file)
    pth2 = file

    if os.path.exists(pth1):
        df = pd.read_csv(pth1, na_values=np.nan)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2, na_values=np.nan)
    else:
        print('Что-то пошло не так')
        return None
        
    try:
        df = df.dropna(subset=['text'])
    except:
        print('Столбец "text" отсутствует в датафрейме')
        
    return df

In [3]:
# Разобьем сообщения на части (чанки) по заданному количеству слов

def split_on_chunks(df, max_words_per_chunk=500):
    current_words = 0
    current_chunk = []
    chunks = []
    
    for index, row in df.iterrows():
        # Считаем количество слов в текущем сообщении
        words_in_message = len(row['text'].split())
        
        # Если сообщение превышает лимит, разбиение на несколько частей
        if words_in_message > max_words_per_chunk:
            # Разбиваем длинное сообщение на несколько частей
            words = row['text'].split()
            while len(words) > max_words_per_chunk:
                part = ' '.join(words[:max_words_per_chunk])
                chunks.append([part])  # Добавляем эту часть как отдельный чанк
                words = words[max_words_per_chunk:]
            # Оставшуюся часть добавляем как последний чанк
            if words:
                chunks.append([' '.join(words)])
        elif current_words + words_in_message > max_words_per_chunk:
            # Если добавление сообщения превысит лимит, добавляем текущий чанк
            chunks.append(current_chunk)
            # Начинаем новый чанк с текущего сообщения
            current_chunk = [row['text']]
            current_words = words_in_message
        else:
            # Добавляем сообщение к текущему чанку
            current_chunk.append(row['text'])
            current_words += words_in_message
    
    # Добавляем последний чанк, если он есть
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

In [4]:
# Общая функция суммаризации текста
def summarize_chunk(chunk, tokenizer, model, prefix=''):
    # Объединяем сообщения части в один текст
    text = ' '.join(chunk)
    
    # Форматируем для модели
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="longest")
    
    # Генерируем резюме
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, min_length=50, length_penalty=2, early_stopping=True)
    
    # Декодируем результат
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summary = summary.replace("summarize: ", "") # убираем "summarize: " для BART
    return summary

In [5]:
# Отдельная функция для ruGPT3
def summarize_with_rugpt3(text, tokenizer, model, max_length=150):
    prompt = f"Текст: {text.strip()}\nКраткое содержание:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)

    output_ids = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_length,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        top_k=50,
        num_beams=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    summary_start = generated.find("Краткое содержание:")
    return generated[summary_start + len("Краткое содержание:"):].strip() if summary_start != -1 else generated


In [6]:
# Функция для расчета сходства
def calculate_similarity(original_text, summary_text):
    # модель SentenceTransformer, и метод encode преобразует текст в числовое представление
    model_st = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    emb_src = model_st.encode(original_text, convert_to_tensor=True) 
    emb_sum = model_st.encode(summary_text, convert_to_tensor=True)
    
    # Cosine similarity
    cos_sim = util.pytorch_cos_sim(emb_src, emb_sum)
    cos_sim_score = cos_sim.item()
    
    return cos_sim_score

### Загрузка данных

In [7]:
df = load_df('data.csv')
df.head()

,date,chat_name,chat_id,sender_id,username,text
2,2025-02-03T11:28:38,💬 Data Practicum Chat,1379846874,user312724902,Olga Varavina,Всем большой привет! Приглашаю на свой уютный ...
3,2025-02-03T11:52:20,💬 Data Practicum Chat,1379846874,user1349934990,Илья,А у тебя когда будет свой канал про аналитику?
4,2025-02-03T11:52:37,💬 Data Practicum Chat,1379846874,user1349934990,Илья,Будешь туда голосовухи пятиминутные постить
5,2025-02-03T11:55:09,💬 Data Practicum Chat,1379846874,user60031833,Sergey,"Потому что сделаны так, будто устарели уже лет..."
6,2025-02-03T11:56:57,💬 Data Practicum Chat,1379846874,user60031833,Sergey,Подкаст?)


In [8]:
chunks = split_on_chunks(df)

### Загрузка моделей

In [9]:
tokenizer_t5 = T5Tokenizer.from_pretrained("google/mt5-small", legacy=False)
model_t5 = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [10]:
tokenizer_bart = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="ru_RU", tgt_lang="ru_RU")
model_bart = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

In [11]:
tokenizer_rut5 = AutoTokenizer.from_pretrained("cointegrated/rut5-small")
model_rut5 = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
tokenizer_gpt3 = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model_gpt3 = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [13]:
tokenizer_rut5_bsg = AutoTokenizer.from_pretrained("IlyaGusev/rut5_base_sum_gazeta")
model_rut5_bsg = AutoModelForSeq2SeqLM.from_pretrained("IlyaGusev/rut5_base_sum_gazeta")

### Суммаризация текста

In [14]:
# Для ускорения вывода и проверки использую срез чанков chunks[1:3], если нужно по всем, убери [1:3]

In [15]:
chunk_summaries_bart = [summarize_chunk(chunk, tokenizer_bart, model_bart) for chunk in chunks[1:3]]

In [16]:
chunk_summaries_t5 = [summarize_chunk(chunk, tokenizer_t5, model_t5, prefix="summarize: ") for chunk in chunks[1:3]]

In [17]:
chunk_summaries_rut5 = [summarize_chunk(chunk, tokenizer_rut5, model_rut5, prefix="summarize: ") for chunk in chunks[1:3]]

In [18]:
chunk_summaries_rut5_bsg = [summarize_chunk(chunk, tokenizer_rut5_bsg, model_rut5_bsg, prefix="summarize: ") for chunk in chunks[1:3]]

In [19]:
chunk_summaries_gpt3 = [summarize_with_rugpt3(' '.join(chunk), tokenizer_gpt3, model_gpt3) for chunk in chunks[1:3]]

In [20]:
chunk_summaries_t5

['<extra_id_0> и т.д.))))))))))))))))))))))))))))))))))))))))))))))))))) ().) )) )) )) )) )) )) )) ))) ))) )) ))) ))) ))) ))) ))))))))))))))))))))))))))))))))))))))',
 '<extra_id_0> не знаю. Интересно, а написать в сопровождение. Инсайты. Можно в любом формате. Может. можно только в любом формате. Может. можно.']

In [21]:
chunk_summaries_bart

['Предложение поучаствовать в бета-тестировании Продуктового аналитика и BI-аналитика *для выпускников курсов "Аналитик данных", "Аналитик данных расширенный" и "Аналитик данных Bootcamp" Всем привет! На связи команда курсов Аналитики данных. Мы активно работаем над курсами Продуктового аналитика и BI-аналитика, и сейчас мы в поиске бета-тестировщиков новых курсов. Для тестирования необходимы навыки в аналитике данных, именно поэтому мы пришли к вам — нашим выпускникам #бета-тест — это возможность бесплатно пройти обучение с сопровождением в Яндекс Практикуме. Главная',
 'я написал в саппорт =D + Все уже обозначили, что ссылка на Nda неактивна? Щас проверю и напишу Не могу подписать NDA "Ссылка неактивна. Запросите новую ссылку"... Давай уже первые подкасты, а то начнутся отписки Отписки начнутся, когда пойдут мои подкасты Подписался Вообще, было бы неплохо, если все участники записывали туда какие- то кейсы рабочие. Инсайты. Можно в любом формате. Написал в сопровождение, но там по пр

In [22]:
chunk_summaries_rut5

['Что нужно делать тестировщику, которая может улучшить процесс обучения, именно поэтому мы пришли к тебе в бета-тестировании по материалам, которая может улучшить процесс учёбы, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы сможем выдать дипломы о прохождении курса, которая может улучшить процесс учёбы',
 'Что может быть неплохо, если все участники записывали туда какие-то кейсы рабочие, Инсайты, а тоже отправила заявку без NDA, как только оставила заявку без NDA, как только оставила заявку без NDA, как только оставила заявку без NDA, как только оставила заявку без ссылки, но не могу подписать отправил заявку, но не могу подписать, как только можно подписать, как только можно подписать, как только можно подписать, как только можно подписать, как только можно подписать,']

In [23]:
chunk_summaries_rut5_bsg

['Выпускники курсов Продуктового аналитика и BI-аналитика получат возможность бесплатно пройти бета-тестирование Продуктового аналитика и BI-аналитика с сопровождением в Яндекс Практикуме. Тестирование будет проходить с середины февраля 2025 по начало июня 2025.',
 'Если все участники курса LLM будут записывать свои годосовухи, то начнутся отписки, когда пойдут мои подкасты, а то начнутся отписки, когда пойдут мои подкасты.']

In [24]:
chunk_summaries_gpt3

['"Исходный код продукта" - это набор инструкций, кодов и инструкторов, который помогает оптимизировать бизнес и производительность бизнес процессов. При разработке продуктов, разработчики обычно используют набор алгоритмов. Каждый алгоритм в свою очередь, является результатом обработки информации от различных источников, таким образом, все эти алгоритмы и инструкции должны быть связаны между собой.\nДанный код описывает последовательность шагов, необходимых для выполнения алгоритма. Код - часть алгоритмической структуры, поэтому он должен быть реализован таким, каким он описан. Алгоритм - набор данных - инструктирующих, управляющих и управляющих, предназначенных для определения последовательности выполнения. В большинстве случаев, алгоритм может быть представлен в двух видах: алгоритм и инструкция. Каждое из этих понятий описывает способ',
 'Мы все учились, мы все работаем.\nВсе ли знают, чего они хотят?\nПсихологический портрет на основании результатов исследования. Выявление сильных

### Метрики для суммаризации

Cosine Similarity (косинусная мера схожести)

In [25]:
results = {
    "Chunk": [],
    "BART": [],
    "T5": [],
    "rut5": [],
    "rut5_bsg": [], 
    "rugpt3": []
}

for i in range(1, 3):
    original_text = ' '.join(chunks[i-1])
    
    summary_bart = chunk_summaries_bart[i-1]
    cos_sim_bart = calculate_similarity(original_text, summary_bart)
    
    summary_t5 = chunk_summaries_t5[i-1]
    cos_sim_t5 = calculate_similarity(original_text, summary_t5)
    
    summary_rut5 = chunk_summaries_rut5[i-1]
    cos_sim_rut5 = calculate_similarity(original_text, summary_rut5)
    
    summary_rut5_bsg = chunk_summaries_rut5_bsg[i-1]
    cos_sim_rut5_bsg = calculate_similarity(original_text, summary_rut5_bsg)
    
    summary_gpt3 = chunk_summaries_gpt3[i-1]
    cos_sim_gpt3 = calculate_similarity(original_text, summary_gpt3)
    
    results["Chunk"].append(f"Chunk {i}")
    results["BART"].append(cos_sim_bart)
    results["T5"].append(cos_sim_t5)
    results["rut5"].append(cos_sim_rut5)
    results["rut5_bsg"].append(cos_sim_rut5_bsg)
    results["rugpt3"].append(cos_sim_gpt3)

df_results = pd.DataFrame(results)

In [26]:
df_results

,Chunk,BART,T5,rut5,rut5_bsg,rugpt3
0,Chunk 1,0.436778,0.225437,0.204232,0.279810,0.266572
1,Chunk 2,0.251637,0.157351,0.178092,0.385551,0.498198


### Вывод
 - Для ускорения вывода и проверки использовал срез из двух кусков(chunks).
 - Попробовал использовать для сумаризации:
     - mT5 - многоязыковую версию T5, предварительно обученную на многоязыковом корпусе Common Crawl (mC4), охватывающем 101 язык;
     - mBART-50	многоязыковую версию BART, предварительно обученную на 50 языках;
     - rut5 - русскоязычную модель на основе модели google/mt5-small;
     - rugpt3large_based_on_gpt2 - архитектура модели основана на GPT-2, но обучение фокусировалось на данных русского языка.
 - mT5 на выхожде не дает осмысленного текста и добавляет специальные токены, такие как, например, 'extra_id_0', не смотря на то, что установил параметр, который должен их убирать skip_special_tokens=True.
 - mBART-50 дала хорошший результат.
 - rut5 дала суммаризацию лучше чем mT5. Но есть многочисленные повторы слов в суммаризованной информации.
 - rugpt3large_based_on_gpt2 - не смотря на мои ожидания, недостаточно хорошо суммаризировала текст и добавляла служебные символы, например, "\n".
 - Cosine Similarity (косинусная мера схожести) показала низкие значения, а чем ниже метрика тем тексты менее схожи. Я получил максимальную оценку на rugpt3 ~0.45 и mBART-50 ~0.44. Остальные значения меньше 0.4 и говорят о том, что тексты могут быть совершенно разными.
 - Использовались небольшие модели, но и на них суммаризация на офисном ноутбуке проиходит очень медленно.
 - Возможно, следует использовать для задач суммаризации нейронки через API, если нет ресурсов для локального использования более мощных моделей.